<a href="https://colab.research.google.com/github/ramachandra742/Machine-Learning-Projects-Jupyter/blob/master/ASHRAE_Great_Energy_Predictor_III_LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load the datasets from kaggle**

In [10]:
from google.colab import files
uploaded_json=files.upload()                  #Upload kaggle API json file

Saving kaggle.json to kaggle.json


In [0]:
!pip install -q kaggle        #install kaggle API

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/         #move kaggle json file to /.kaggle

In [13]:
!ls                    #list directory

kaggle.json  sample_data


In [0]:

!chmod 600 /root/.kaggle/kaggle.json

In [15]:
!kaggle competitions download -c ashrae-energy-prediction

 97% 161M/167M [00:03<00:00, 61.0MB/s]
100% 167M/167M [00:03<00:00, 56.9MB/s]
  0% 0.00/2.53M [00:00<?, ?B/s]
100% 2.53M/2.53M [00:00<00:00, 171MB/s]
 94% 113M/120M [00:02<00:00, 41.7MB/s]
100% 120M/120M [00:02<00:00, 47.0MB/s]
  0% 0.00/1.27M [00:00<?, ?B/s]
100% 1.27M/1.27M [00:00<00:00, 179MB/s]
 94% 83.0M/88.4M [00:01<00:00, 50.4MB/s]
100% 88.4M/88.4M [00:01<00:00, 50.9MB/s]
  0% 0.00/44.5k [00:00<?, ?B/s]
100% 44.5k/44.5k [00:00<00:00, 39.7MB/s]


In [16]:
!unzip \*.zip

Archive:  weather_train.csv.zip
  inflating: weather_train.csv       

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

Archive:  weather_test.csv.zip
  inflating: weather_test.csv        

5 archives were successfully processed.


Read the datasets

In [0]:
#Selected columns 
metadata_cols=['site_id','building_id','primary_use','square_feet','floor_count']
weather_cols=['site_id','timestamp','air_temperature','dew_temperature','sea_level_pressure','wind_direction','wind_speed']

In [0]:
import pandas as pd
train_df=pd.read_csv('train.csv',na_values='NaN')
test_df=pd.read_csv('test.csv',na_values='NaN')
weather_train=pd.read_csv('weather_train.csv',na_values='NaN',usecols=weather_cols)
weather_test=pd.read_csv('weather_test.csv',na_values='NaN',usecols=weather_cols)
building_metadata=pd.read_csv('building_metadata.csv',na_values='NaN',usecols=metadata_cols)

Lets explore the datasets

In [19]:
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [20]:
train_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int64
meter            int64
timestamp        object
meter_reading    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.9 GB


In [21]:
test_df.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00


In [22]:
test_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 4 columns):
row_id         int64
building_id    int64
meter          int64
timestamp      object
dtypes: int64(3), object(1)
memory usage: 3.9 GB


In [23]:
weather_train.head()

,site_id,timestamp,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,20.0,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,21.1,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,21.1,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,20.6,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,20.0,1020.0,250.0,2.6


In [24]:
weather_train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 7 columns):
site_id               139773 non-null int64
timestamp             139773 non-null object
air_temperature       139718 non-null float64
dew_temperature       139660 non-null float64
sea_level_pressure    129155 non-null float64
wind_direction        133505 non-null float64
wind_speed            139469 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 16.5 MB


In [25]:
weather_test.head()

,site_id,timestamp,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.8,11.7,1021.4,100.0,3.6
1,0,2017-01-01 01:00:00,17.8,12.8,1022.0,130.0,3.1
2,0,2017-01-01 02:00:00,16.1,12.8,1021.9,140.0,3.1
3,0,2017-01-01 03:00:00,17.2,13.3,1022.2,140.0,3.1
4,0,2017-01-01 04:00:00,16.7,13.3,1022.3,130.0,2.6


In [26]:
weather_test.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277243 entries, 0 to 277242
Data columns (total 7 columns):
site_id               277243 non-null int64
timestamp             277243 non-null object
air_temperature       277139 non-null float64
dew_temperature       276916 non-null float64
sea_level_pressure    255978 non-null float64
wind_direction        264873 non-null float64
wind_speed            276783 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 32.8 MB


In [27]:
building_metadata.head()

,site_id,building_id,primary_use,square_feet,floor_count
0,0,0,Education,7432,NaN
1,0,1,Education,2720,NaN
2,0,2,Education,5376,NaN
3,0,3,Education,23685,NaN
4,0,4,Education,116607,NaN


In [28]:
building_metadata.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 5 columns):
site_id        1449 non-null int64
building_id    1449 non-null int64
primary_use    1449 non-null object
square_feet    1449 non-null int64
floor_count    355 non-null float64
dtypes: float64(1), int64(3), object(1)
memory usage: 144.2 KB


In [29]:
#Prepare train dataset

df_train=train_df.merge(building_metadata,on='building_id',how='left')
df_train=df_train.merge(weather_train,on=['site_id','timestamp'],how='left')
df_train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,floor_count,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,NaN,25.0,20.0,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,NaN,25.0,20.0,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,NaN,25.0,20.0,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,NaN,25.0,20.0,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,NaN,25.0,20.0,1019.7,0.0,0.0


In [0]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df_train['primary_use']=le.fit_transform(df_train['primary_use'])

In [31]:
df_train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20216100 entries, 0 to 20216099
Data columns (total 13 columns):
building_id           int64
meter                 int64
timestamp             object
meter_reading         float64
site_id               int64
primary_use           int64
square_feet           int64
floor_count           float64
air_temperature       float64
dew_temperature       float64
sea_level_pressure    float64
wind_direction        float64
wind_speed            float64
dtypes: float64(7), int64(5), object(1)
memory usage: 3.4 GB


In [32]:
import gc
del train_df,weather_train
gc.collect()

0

**Reduce the memory usage for train dataset**


In [0]:
import numpy as np
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            print("min for this col: ",mn)
            print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

In [34]:
df_train,NAlist=reduce_mem_usage(df_train)

Memory usage of properties dataframe is : 2159.3124389648438  MB
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype after:  uint8
******************************
******************************
Column:  meter_reading
dtype before:  float64
min for this col:  0.0
max for this col:  21904700.0
dtype after:  float32
******************************
******************************
Column:  site_id
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  primary_use
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  square_feet
dtype before:  int64
min for this col:  

In [0]:
#convert timestamp to datetime for df_train
df_train['timestamp']=pd.to_datetime(df_train['timestamp'])
df_train['hour']=df_train['timestamp'].dt.hour
df_train['day']=df_train['timestamp'].dt.day
df_train['month']=df_train['timestamp'].dt.month
df_train['weekend']=df_train['timestamp'].dt.weekday
df_train['year']=df_train['timestamp'].dt.year

In [0]:
df_train=df_train.drop(columns='timestamp',axis=1)

In [0]:
#Replace missing value with -999
df_train=df_train.fillna(-999)

In [38]:
df_train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,floor_count,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,hour,day,month,weekend,year
0,0,0,0.0,0,0,7432,0,25.0,20.0,1019.700012,0,0.0,0,1,1,4,2016
1,1,0,0.0,0,0,2720,0,25.0,20.0,1019.700012,0,0.0,0,1,1,4,2016
2,2,0,0.0,0,0,5376,0,25.0,20.0,1019.700012,0,0.0,0,1,1,4,2016
3,3,0,0.0,0,0,23685,0,25.0,20.0,1019.700012,0,0.0,0,1,1,4,2016
4,4,0,0.0,0,0,116607,0,25.0,20.0,1019.700012,0,0.0,0,1,1,4,2016


In [0]:
#consider 50% of train data points randomly
#rain=df_train.sample(frac=0.5)

In [40]:
original_labels=np.log1p(df_train['meter_reading'])
target_labels=df_train['meter_reading']
target_labels.shape

(20216100,)

In [0]:
df_train=df_train.drop('meter_reading',axis=1)

In [42]:
#prepare test dataset
df_test=test_df.merge(building_metadata,on='building_id',how='left')
df_test=df_test.merge(weather_test,on=['site_id','timestamp'],how='left')
df_test.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,floor_count,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,NaN,17.8,11.7,1021.4,100.0,3.6
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,NaN,17.8,11.7,1021.4,100.0,3.6
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,NaN,17.8,11.7,1021.4,100.0,3.6
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,NaN,17.8,11.7,1021.4,100.0,3.6
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,NaN,17.8,11.7,1021.4,100.0,3.6


In [0]:
#df_test=df_test.fillna(-999)

In [44]:
df_test,NAlist=reduce_mem_usage(df_test)

Memory usage of properties dataframe is : 4453.7841796875  MB
******************************
Column:  row_id
dtype before:  int64
min for this col:  0
max for this col:  41697599
dtype after:  uint32
******************************
******************************
Column:  building_id
dtype before:  int64
min for this col:  0
max for this col:  1448
dtype after:  uint16
******************************
******************************
Column:  meter
dtype before:  int64
min for this col:  0
max for this col:  3
dtype after:  uint8
******************************
******************************
Column:  site_id
dtype before:  int64
min for this col:  0
max for this col:  15
dtype after:  uint8
******************************
******************************
Column:  square_feet
dtype before:  int64
min for this col:  283
max for this col:  875000
dtype after:  uint32
******************************
******************************
Column:  floor_count
dtype before:  float64
min for this col:  1.0
max 

In [0]:
#transform object to categorical class
df_test['primary_use']=le.transform(df_test['primary_use'])

In [0]:
#convert timestamp to datetime for df_test
df_test['timestamp']=pd.to_datetime(df_test['timestamp'])
df_test['hour']=df_test['timestamp'].dt.hour
df_test['day']=df_test['timestamp'].dt.day
df_test['month']=df_test['timestamp'].dt.month
df_test['weekend']=df_test['timestamp'].dt.weekday
df_test['year']=df_test['timestamp'].dt.year

In [0]:
df_test=df_test.drop('timestamp',axis=1)

In [48]:
#create train & validation datasets
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(df_train,target_labels,test_size=0.1,random_state=42)
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)

(18194490, 16) (18194490,)
(2021610, 16) (2021610,)


In [0]:
#Build the lightgbm model
import lightgbm as lgb
import gc
train_data=lgb.Dataset(X_train,label=y_train)
validation_data=lgb.Dataset(X_val,label=y_val)


In [0]:
params={ 'obective':'regression',
         'num_leaves':125,
         'metric':'mae',
         'num_rounds':1000,
         'learning_rate':0.12,
         'min_data_per_leaf':20,
         'lamda_l2':0.01    
}

In [51]:
lgb_model=lgb.train(params,train_data,valid_sets=validation_data,early_stopping_rounds=100)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 3407.05
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's l1: 3068.41
[3]	valid_0's l1: 2775.38
[4]	valid_0's l1: 2513.46
[5]	valid_0's l1: 2287.45
[6]	valid_0's l1: 2091.8
[7]	valid_0's l1: 1924.54
[8]	valid_0's l1: 1777.83
[9]	valid_0's l1: 1647.8
[10]	valid_0's l1: 1536.49
[11]	valid_0's l1: 1441.4
[12]	valid_0's l1: 1355.84
[13]	valid_0's l1: 1284.94
[14]	valid_0's l1: 1224.11
[15]	valid_0's l1: 1171.2
[16]	valid_0's l1: 1125.53
[17]	valid_0's l1: 1086.9
[18]	valid_0's l1: 1045.27
[19]	valid_0's l1: 1015.73
[20]	valid_0's l1: 991.846
[21]	valid_0's l1: 970.683
[22]	valid_0's l1: 939.482
[23]	valid_0's l1: 915.149
[24]	valid_0's l1: 893.481
[25]	valid_0's l1: 871.43
[26]	valid_0's l1: 850.994
[27]	valid_0's l1: 838.448
[28]	valid_0's l1: 821.844
[29]	valid_0's l1: 811.156
[30]	valid_0's l1: 796.142
[31]	valid_0's l1: 783.448
[32]	valid_0's l1: 769.676
[33]	valid_0's l1: 757.095
[34]	valid_0's l1: 746.845
[35]	valid_0's l1: 741.16
[36]	val

In [52]:
lgb_model.save_model('lgb_model.txt')

In [0]:
lgb_json=lgb_model.dump_model()

In [0]:
y_predict=lgb_model.predict(df_test)

In [55]:
lgb=pd.read_csv('sample_submission.csv')
lgb.head()

,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [0]:
lgb['meter_reading']=y_predict

In [57]:
lgb,NAlist=reduce_mem_usage(lgb)

Memory usage of properties dataframe is : 636.2550048828125  MB
******************************
Column:  row_id
dtype before:  int64
min for this col:  0
max for this col:  41697599
dtype after:  uint32
******************************
******************************
Column:  meter_reading
dtype before:  float64
min for this col:  -526.3532351150725
max for this col:  53982.07161826538
dtype after:  float32
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  318.1275634765625  MB
This is  50.00000959287641 % of the initial size


In [58]:
lgb.head()

,row_id,meter_reading
0,0,-329.645721
1,1,1378.805908
2,2,1680.924805
3,3,787.752991
4,4,787.752991


In [0]:
lgb.to_csv('lgb_submission_2.csv',index=False)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cp lgb_submission_2.csv '/content/drive/My Drive/Colab Notebooks'